In [ ]:
#!pip install tokenizers==0.9.4
#!pip install transformers==4.2.1
import sys
#sys.modules['tokenizers'].__version__


In [ ]:

import importlib
importlib.reload(sys.modules['pkg_resources'])

sys.modules['pkg_resources'].get_distribution('tokenizers').version


In [ ]:

from numba import cuda 
import pandas as pd

import re
import os
import numpy as np
import pandas as pd
import random
import math
import tensorflow as tf
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from tensorflow.keras import backend as K
from transformers import RobertaTokenizer, TFRobertaModel
from kaggle_datasets import KaggleDatasets
tf.get_logger().setLevel(logging.ERROR)
from kaggle_datasets import KaggleDatasets
from transformers import AutoTokenizer

import numpy as np
import pandas as pd 
import os
import gc
import sys
import time
import random
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold, StratifiedKFold
import xgboost as xgb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import mean_squared_error

from transformers import AutoModel, AutoTokenizer
import json
from tensorflow.keras.models import load_model
import re
import pandas as pd
import string
import keras
from sklearn.svm import SVR

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import gc

from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression, Ridge

import tensorflow as tf 
from tensorflow.keras.layers import Input,LSTM,Bidirectional,Embedding,Dense, Conv1D, Dropout , MaxPool1D , MaxPooling1D, GlobalAveragePooling2D , GlobalAveragePooling1D , GlobalMaxPooling1D , concatenate , Flatten
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.models import Model,load_model,save_model , model_from_json
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint, EarlyStopping ,LearningRateScheduler
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

from transformers import TFBertModel, BertTokenizerFast , BertTokenizer , RobertaTokenizerFast , TFRobertaModel , RobertaConfig , TFAutoModel , AutoTokenizer


In [ ]:
import sys
sys.modules['tokenizers'].__version__

In [ ]:
import importlib
importlib.reload(sys.modules['pkg_resources'])

sys.modules['pkg_resources'].get_distribution('tokenizers').version

In [ ]:
tf.__version__

In [ ]:
import transformers
transformers.__version__

In [ ]:
!ls '../input/commonlitdistilroberta'


# Model 0 :
https://www.kaggle.com/bharadwajvedula/clr-lb-0-475-lazy-way-to-get-good-score


In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu,True)
        
max_len = 250
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

MODEL=['bert-base-uncased' , 'roberta-base']

model_name = MODEL[1]

path=[
    "../input/commonlitreadabilityprize/sample_submission.csv",
    "../input/commonlitreadabilityprize/test.csv",
    "../input/commonlitreadabilityprize/train.csv"
]

df_train = pd.read_csv(path[2])
df_test = pd.read_csv(path[1])
df_ss = pd.read_csv(path[0])
                         
df_train = df_train.drop(['url_legal','license','standard_error'],axis='columns')
df_test = df_test.drop(['url_legal','license'],axis='columns')
X= df_train['excerpt']
y=df_train['target'].values

X_test = df_test['excerpt']

tokenizer1 = AutoTokenizer.from_pretrained("../input/huggingface-roberta-variants/roberta-base/roberta-base")

print('tokenization')
train_embeddings = tokenizer1(X.to_list(), truncation = True , padding = 'max_length' , max_length=max_len)
test_embeddings = tokenizer1(X_test.to_list() , truncation = True , padding = 'max_length' , max_length = max_len)
                         
@tf.function
def map_function(encodings):
    input_ids = encodings['input_ids']
    
    return {'input_word_ids': input_ids}

print("generating train and test")    
train = tf.data.Dataset.from_tensor_slices((train_embeddings))
train = (
            train
            .map(map_function, num_parallel_calls=AUTOTUNE)
            .batch(16)
            .prefetch(AUTOTUNE)
        )


test = tf.data.Dataset.from_tensor_slices((test_embeddings))
test = (
        test
        .map(map_function, num_parallel_calls = AUTOTUNE)
        .batch(16)
        .prefetch(AUTOTUNE)
    )
                         
                         
def build_roberta_base_model(max_len=max_len ):
    
    transformer = TFAutoModel.from_pretrained("../input/huggingface-roberta-variants/roberta-base/roberta-base")
    
    input_word_ids = tf.keras.layers.Input(shape = (max_len, ), dtype = tf.int32, name = 'input_word_ids')
    sequence_output = transformer(input_word_ids)[0]
    
    # We only need the cls_token, resulting in a 2d array
    cls_token = sequence_output[:, 0, :]
    output = tf.keras.layers.Dense(1, activation = 'linear', dtype = 'float32')(cls_token)
    
    model = tf.keras.models.Model(inputs = [input_word_ids], outputs = output)
    
    return model
                         
ragnar_model = build_roberta_base_model()
def feature_extractor(path):
    print("loading weights")
    ragnar_model.load_weights(path)
    x= ragnar_model.layers[-3].output
    model = Model(inputs = ragnar_model.inputs , outputs = x)
    return model
                         
def get_preds(model,train,test):
    print("Extracting Features from train data")
    train_features = model.predict( train , verbose =1)
    train_features = train_features.last_hidden_state
    train_features = train_features[: , 0 , :]
    print("Extracting Features from train data")
    test_features = model.predict( test , verbose =1)
    test_features = test_features.last_hidden_state
    test_features = test_features[: , 0 , :]
    
    return np.array(train_features , dtype= np.float16) , np.array(test_features , dtype= np.float16) 
                         
#model weight paths
paths=["../input/commonlit-readability-roberta-base/Roberta_Base_123_1.h5",
       "../input/commonlit-readability-roberta-base/Roberta_Base_123_2.h5",
       "../input/commonlit-readability-roberta-base/Roberta_Base_123_3.h5",
       "../input/commonlit-readability-roberta-base/Roberta_Base_123_4.h5",
       "../input/commonlit-readability-roberta-base/Roberta_Base_123_5.h5"
      ]
                         
#1
extraction_model = feature_extractor(paths[0])
train_embeddings1 , test_embeddings1 = get_preds(extraction_model , train , test)
                         
#2
extraction_model = feature_extractor(paths[1])
train_embeddings2 , test_embeddings2 = get_preds(extraction_model , train , test)
                         
#3
extraction_model = feature_extractor(paths[2])
train_embeddings3 , test_embeddings3 = get_preds(extraction_model , train , test)
                         
#4
extraction_model = feature_extractor(paths[3])
train_embeddings4 , test_embeddings4 = get_preds(extraction_model , train , test)
                         
#5
extraction_model = feature_extractor(paths[4])
train_embeddings5 , test_embeddings5 = get_preds(extraction_model , train , test)
                         
def get_preds(train_embeddings , test_embeddings):
    scores=[]
    kfold = KFold(n_splits=5, shuffle= True , random_state=2021)
    iteration=1
    preds = np.zeros((test_embeddings.shape[0]))
    for train_idx, test_idx in kfold.split(train_embeddings,y):
        print(f'running iteration {iteration}')
        X_train = train_embeddings[train_idx]
        X_test = train_embeddings[test_idx]
        y_train = y[train_idx]
        y_test = y[test_idx]

        regression_model = Ridge()
        
        regression_model.fit(X_train,y_train)
        y_pred = regression_model.predict(X_test)

        score = np.sqrt(mse(y_pred,y_test))
        scores.append(score)
        print(f'Fold {iteration} , rmse score: {score}')
        y_preds = regression_model.predict(test_embeddings)
        y_preds=y_preds.reshape(-1)
        preds+=y_preds  
        iteration += 1

    print(f"the average rmse is {np.mean(scores)}")
    return np.array(preds)/5  
                         
                         
print("***********predicting***********")
preds1 = get_preds(train_embeddings1,test_embeddings1)
print("***********predicting***********")
preds2 = get_preds(train_embeddings2,test_embeddings2)
print("***********predicting***********")
preds3 = get_preds(train_embeddings3,test_embeddings3)
print("***********predicting***********")
preds4 = get_preds(train_embeddings4,test_embeddings4)
print("***********predicting***********")
preds5 = get_preds(train_embeddings5,test_embeddings5)

preds=(preds1+preds2+preds3+preds4+preds5)/5
preds = preds.tolist()
sub0=pd.DataFrame({'id':df_ss['id'],'target':preds})
#sub.to_csv('submission.csv',index=False)
sub0.head()

# model 1 : 
https://www.kaggle.com/mobassir/readability-tensorflow-roberta-training-baseline

In [ ]:
# Read our test data
df = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

In [ ]:
#generator = AutoTokenizer.from_pretrained(MODEL,  use_fast=True) 

In [ ]:
os.listdir('../input/readability-tensorflow-roberta-training-baseline/')

In [ ]:
models = [
    
# fold 1 -> 0.50380
#fold 2 -> 0.51644
#fold 3 -> 0.49793
#fold 4 -> 0.53571 
#fold 5 -> 0.50805
    
    '../input/commonlitdistilroberta/Roberta_distilroberta_stratifiedkfold_12345_5.h5',
    #'../input/commonlitdistilroberta/Roberta_distilroberta_stratifiedkfold_12345_4.h5',
    '../input/commonlitdistilroberta/Roberta_distilroberta_stratifiedkfold_12345_3.h5',
    #'../input/commonlitdistilroberta/Roberta_distilroberta_stratifiedkfold_12345_2.h5',
    '../input/commonlitdistilroberta/Roberta_distilroberta_stratifiedkfold_12345_1.h5',
  
  
]

In [ ]:
# This function tokenize the text according to a transformers model tokenizer
def regular_encode(texts, tokenizer, maxlen = 350):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        padding = 'max_length',
        truncation = True,
        max_length = maxlen,
    )
    
    return np.array(enc_di['input_ids'])

# This function encode our training sentences
def encode_texts(x_test, MAX_LEN = 350):
    x_test = regular_encode(x_test.tolist(), tokenizer, maxlen = MAX_LEN)
    return x_test

# Function to build our model
def build_roberta_model(max_len = 350,MODEL=None):
    transformer = TFRobertaModel.from_pretrained(MODEL)
    input_word_ids = tf.keras.layers.Input(shape = (max_len, ), dtype = tf.int32, name = 'input_word_ids')
    sequence_output = transformer(input_word_ids)[0]
    # We only need the cls_token, resulting in a 2d array
    cls_token = sequence_output[:, 0, :]
    output = tf.keras.layers.Dense(1, activation = 'linear', dtype = 'float32')(cls_token)
    model = tf.keras.models.Model(inputs = [input_word_ids], outputs = [output])
    return model

# Function for inference
def distilroberta_base_inference():
    # Configurations

    # Max length
    MAX_LEN = 350
    # Get the trained model we want to use

    MODEL = '../input/huggingface-roberta-variants/distilroberta-base/distilroberta-base'

    #MODEL = '../input/distilrobertapretrained'

    # Let's load our model tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    #tokenizer = AutoTokenizer.from_pretrained("distilroberta-base") 
    

    # Get text features
    x_test = df['excerpt']
    # Encode our text with Roberta tokenizer
    x_test = encode_texts(x_test, MAX_LEN)
    # Initiate an empty vector to store prediction
    predictions = np.zeros(len(df))
    # Predict with the 5 models (5 folds training)
    for i in range(len(models)):
        print('\n')
        print('-'*50)
        print(f'Predicting with model {i + 1}')
        # Build model
        model = build_roberta_model(max_len = MAX_LEN,MODEL = MODEL)
        # Load pretrained weights
        print(models[i])
        model.load_weights(models[i])
        # Predict
        fold_predictions = model.predict(x_test).reshape(-1)
        # Add fold prediction to the global predictions
        predictions += fold_predictions / len(models)
    # Save submissions
    #df['target'] = predictions
    #df[['id', 'target']].to_csv('submissiontensorflow.csv', index = False)
    print("donee")
    return predictions

#tfdf = distilroberta_base_inference()

# model 2:
https://www.kaggle.com/ragnar123/commonlit-readability-roberta-tf-inference

In [ ]:
# Configurations
# Number of folds for training
FOLDS = 5
# Max length
MAX_LEN = 250
# Get the trained model we want to use
MODEL = '../input/tfroberta-base'
# Let's load our model tokenizer
tokenizer = RobertaTokenizer.from_pretrained(MODEL)


# This function tokenize the text according to a transformers model tokenizer
def regular_encode(texts, tokenizer, maxlen = MAX_LEN):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        padding = 'max_length',
        truncation = True,
        max_length = maxlen,
    )
    
    return np.array(enc_di['input_ids'])

# This function encode our training sentences
def encode_texts(x_test, MAX_LEN = 350):
    x_test = regular_encode(x_test.tolist(), tokenizer, maxlen = MAX_LEN)
    return x_test

# Function to build our model
def build_roberta_base_model(max_len = MAX_LEN):
    transformer = TFRobertaModel.from_pretrained(MODEL)
    input_word_ids = tf.keras.layers.Input(shape = (max_len, ), dtype = tf.int32, name = 'input_word_ids')
    sequence_output = transformer(input_word_ids)[0]
    # We only need the cls_token, resulting in a 2d array
    cls_token = sequence_output[:, 0, :]
    output = tf.keras.layers.Dense(1, activation = 'linear', dtype = 'float32')(cls_token)
    model = tf.keras.models.Model(inputs = [input_word_ids], outputs = [output])
    return model

# Function for inference
def roberta_base_inference():
    #predictions1 = distilroberta_base_inference()
    # Read our test data
    df = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
    # Get text features
    x_test = df['excerpt']
    # Encode our text with Roberta tokenizer
    x_test = encode_texts(x_test, MAX_LEN)
    # Initiate an empty vector to store prediction
    predictions = np.zeros(len(df))
    # Predict with the 5 models (5 folds training)
    for i in range(FOLDS):
        print('\n')
        print('-'*50)
        print(f'Predicting with model {i + 1}')
        # Build model
        model = build_roberta_base_model(max_len = MAX_LEN)
        # Load pretrained weights
        model.load_weights(f'../input/commonlit-readability-roberta-base/Roberta_Base_123_{i + 1}.h5')
        # Predict
        fold_predictions = model.predict(x_test).reshape(-1)
        # Add fold prediction to the global predictions
        predictions += fold_predictions / FOLDS
    # Save submissions
    
    
    df['target'] = predictions
    #df[['id', 'target']].to_csv('submission2.csv', index = False)
    return df

ragdf = roberta_base_inference()
ragdf.head()

In [ ]:
device = cuda.get_current_device()
device.reset()
!nvidia-smi

# torch

# model 3: https://www.kaggle.com/rhtsingh/commonlit-readability-prize-roberta-torch-infer-3

In [ ]:
%matplotlib inline
from glob import glob
import os
import matplotlib.pyplot as plt
import json
from collections import defaultdict
import gc
gc.enable()
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.optimizer import Optimizer
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import (
    Dataset, DataLoader, 
    SequentialSampler, RandomSampler
)
from transformers import RobertaConfig
from transformers import (
    get_cosine_schedule_with_warmup, 
    get_cosine_with_hard_restarts_schedule_with_warmup
)
from transformers import RobertaTokenizer
from transformers import RobertaModel
from IPython.display import clear_output
from tqdm import tqdm, trange

import pandas as pd
import numpy as np

import os
import gc
import sys
import cv2
import math
import time
import tqdm
import random
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,StratifiedKFold

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import Adam, lr_scheduler
from torch.utils.data import Dataset, DataLoader

from transformers import (AutoModel, AutoTokenizer, 
                          AutoModelForSequenceClassification)

import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff


from colorama import Fore, Back, Style


train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

In [ ]:
def convert_examples_to_features(data, tokenizer, max_len, is_test=False):
    data = data.replace('\n', '')
    tok = tokenizer.encode_plus(
        data, 
        max_length=max_len, 
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=True
    )
    curr_sent = {}
    padding_length = max_len - len(tok['input_ids'])
    curr_sent['input_ids'] = tok['input_ids'] + ([0] * padding_length)
    curr_sent['token_type_ids'] = tok['token_type_ids'] + \
        ([0] * padding_length)
    curr_sent['attention_mask'] = tok['attention_mask'] + \
        ([0] * padding_length)
    return curr_sent

In [ ]:
class DatasetRetriever(Dataset):
    def __init__(self, data, tokenizer, max_len, is_test=False):
        self.data = data
        self.excerpts = self.data.excerpt.values.tolist()
        self.tokenizer = tokenizer
        self.is_test = is_test
        self.max_len = max_len
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, item):
        if not self.is_test:
            excerpt, label = self.excerpts[item], self.targets[item]
            features = convert_examples_to_features(
                excerpt, self.tokenizer, 
                self.max_len, self.is_test
            )
            return {
                'input_ids':torch.tensor(features['input_ids'], dtype=torch.long),
                'token_type_ids':torch.tensor(features['token_type_ids'], dtype=torch.long),
                'attention_mask':torch.tensor(features['attention_mask'], dtype=torch.long),
                'label':torch.tensor(label, dtype=torch.double),
            }
        else:
            excerpt = self.excerpts[item]
            features = convert_examples_to_features(
                excerpt, self.tokenizer, 
                self.max_len, self.is_test
            )
            return {
                'input_ids':torch.tensor(features['input_ids'], dtype=torch.long),
                'token_type_ids':torch.tensor(features['token_type_ids'], dtype=torch.long),
                'attention_mask':torch.tensor(features['attention_mask'], dtype=torch.long),
            }
     

In [ ]:
class CommonLitModel(nn.Module):
    def __init__(
        self, 
        model_name, 
        config,  
        multisample_dropout=False,
        output_hidden_states=False
    ):
        super(CommonLitModel, self).__init__()
        self.config = config
        self.roberta = RobertaModel.from_pretrained(
            model_name, 
            output_hidden_states=output_hidden_states
        )
        self.layer_norm = nn.LayerNorm(config.hidden_size)
        if multisample_dropout:
            self.dropouts = nn.ModuleList([
                nn.Dropout(0.5) for _ in range(5)
            ])
        else:
            self.dropouts = nn.ModuleList([nn.Dropout(0.3)])
        self.regressor = nn.Linear(config.hidden_size, 1)
        self._init_weights(self.layer_norm)
        self._init_weights(self.regressor)
 
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
 
    def forward(
        self, 
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None
    ):
        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        sequence_output = outputs[1]
        sequence_output = self.layer_norm(sequence_output)
 
        # multi-sample dropout
        for i, dropout in enumerate(self.dropouts):
            if i == 0:
                logits = self.regressor(dropout(sequence_output))
            else:
                logits += self.regressor(dropout(sequence_output))
        
        logits /= len(self.dropouts)
 
        # calculate loss
        loss = None
        if labels is not None:
            loss_fn = torch.nn.MSELoss()
            logits = logits.view(-1).to(labels.dtype)
            loss = torch.sqrt(loss_fn(logits, labels.view(-1)))
        
        output = (logits,) + outputs[1:]
        return ((loss,) + output) if loss is not None else output

In [ ]:
def make_model(model_name, num_labels=1):
    tokenizer = RobertaTokenizer.from_pretrained(model_name)
    config = RobertaConfig.from_pretrained(model_name)
    config.update({'num_labels':num_labels})
    model = CommonLitModel(model_name, config=config)
    return model, tokenizer

def make_loader(
    data, 
    tokenizer, 
    max_len,
    batch_size,
):
    
    test_dataset = DatasetRetriever(data, tokenizer, max_len, is_test=True)
    test_sampler = SequentialSampler(test_dataset)
    test_loader = DataLoader(
        test_dataset, 
        batch_size=batch_size // 2, 
        sampler=test_sampler, 
        pin_memory=False, 
        drop_last=False, 
        num_workers=0
    )

    return test_loader

In [ ]:
class Evaluator:
    def __init__(self, model, scalar=None):
        self.model = model
        self.scalar = scalar

    def evaluate(self, data_loader, tokenizer):
        preds = []
        self.model.eval()
        total_loss = 0
        with torch.no_grad():
            for batch_idx, batch_data in enumerate(data_loader):
                input_ids, attention_mask, token_type_ids = batch_data['input_ids'], \
                    batch_data['attention_mask'], batch_data['token_type_ids']
                input_ids, attention_mask, token_type_ids = input_ids.cuda(), \
                    attention_mask.cuda(), token_type_ids.cuda()
                
                if self.scalar is not None:
                    with torch.cuda.amp.autocast():
                        outputs = self.model(
                            input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids
                        )
                else:
                    outputs = self.model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids
                    )
                
                logits = outputs[0].detach().cpu().numpy().squeeze().tolist()
                preds += logits
        return preds

In [ ]:
 def config(fold, model_name, load_model_path):
    torch.manual_seed(2021)
    torch.cuda.manual_seed(2021)
    torch.cuda.manual_seed_all(2021)
    
    max_len = 250
    batch_size = 8

    model, tokenizer = make_model(
        model_name=model_name, 
        num_labels=1
    )
    model.load_state_dict(
        torch.load(f'{load_model_path}/model{fold}.bin')
    )
    test_loader = make_loader(
        test, tokenizer, max_len=max_len,
        batch_size=batch_size
    )

    if torch.cuda.device_count() >= 1:
        print('Model pushed to {} GPU(s), type {}.'.format(
            torch.cuda.device_count(), 
            torch.cuda.get_device_name(0))
        )
        model = model.cuda() 
    else:
        raise ValueError('CPU training is not supported')

    # scaler = torch.cuda.amp.GradScaler()
    scaler = None
    return (
        model, tokenizer, 
        test_loader, scaler
    )

In [ ]:
def run(fold=0, model_name=None, load_model_path=None):
    model, tokenizer, \
        test_loader, scaler = config(fold, model_name, load_model_path)
    
    import time

    evaluator = Evaluator(model, scaler)

    test_time_list = []

    torch.cuda.synchronize()
    tic1 = time.time()

    preds = evaluator.evaluate(test_loader, tokenizer)

    torch.cuda.synchronize()
    tic2 = time.time() 
    test_time_list.append(tic2 - tic1)
    
    del model, tokenizer, test_loader, scaler
    gc.collect()
    torch.cuda.empty_cache()
    
    return preds

In [ ]:
%%time
pred_df1 = pd.DataFrame()
pred_df2 = pd.DataFrame()
pred_df3 = pd.DataFrame()
for fold in tqdm(range(5)):
    pred_df1[f'fold{fold}'] = run(fold, '../input/roberta-base/', '../input/commonlit-roberta-base-i/')
    pred_df2[f'fold{fold+5}'] = run(fold, '../input/robertalarge/', '../input/roberta-large-itptfit/')
    pred_df3[f'fold{fold+10}'] = run(fold, '../input/robertalarge/', '../input/commonlit-roberta-large-ii/')

# model 4 : https://www.kaggle.com/maunish/clrp-roberta-svm

In [ ]:

y_ = Fore.YELLOW
r_ = Fore.RED
g_ = Fore.GREEN
b_ = Fore.BLUE
m_ = Fore.MAGENTA
c_ = Fore.CYAN
sr_ = Style.RESET_ALL


train_data = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test_data = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
sample = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')

num_bins = int(np.floor(1 + np.log2(len(train_data))))
train_data.loc[:,'bins'] = pd.cut(train_data['target'],bins=num_bins,labels=False)

target = train_data['target'].to_numpy()
bins = train_data.bins.to_numpy()

def rmse_score(y_true,y_pred):
    return np.sqrt(mean_squared_error(y_true,y_pred))


config = {
    'batch_size':128,
    'max_len':256,
    'nfolds':5,
    'seed':42,
}

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(seed=config['seed'])


class CLRPDataset(Dataset):
    def __init__(self,df,tokenizer):
        self.excerpt = df['excerpt'].to_numpy()
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],return_tensors='pt',
                                max_length=config['max_len'],
                                padding='max_length',truncation=True)
        return encode
    
    def __len__(self):
        return len(self.excerpt)
    
    
    
class AttentionHead(nn.Module):
    def __init__(self, in_features, hidden_dim, num_targets):
        super().__init__()
        self.in_features = in_features
        self.middle_features = hidden_dim

        self.W = nn.Linear(in_features, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
        self.out_features = hidden_dim

    def forward(self, features):
        att = torch.tanh(self.W(features))

        score = self.V(att)

        attention_weights = torch.softmax(score, dim=1)

        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=1)

        return context_vector
    
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.roberta = AutoModel.from_pretrained('../input/roberta-base')    
        self.head = AttentionHead(768,768,1)
        self.dropout = nn.Dropout(0.1)
        self.linear = nn.Linear(self.head.out_features,1)

    def forward(self,**xb):
        x = self.roberta(**xb)[0]
        x = self.head(x)
        return x
    
    
def get_embeddings(df,path,plot_losses=True, verbose=True):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"{device} is used")
            
    model = Model()
    model.load_state_dict(torch.load(path))
    model.to(device)
    model.eval()
    
    tokenizer = AutoTokenizer.from_pretrained('../input/roberta-base')
    
    ds = CLRPDataset(df,tokenizer)
    dl = DataLoader(ds,
                  batch_size = config["batch_size"],
                  shuffle=False,
                  num_workers = 4,
                  pin_memory=True,
                  drop_last=False
                 )
        
    embeddings = list()
    with torch.no_grad():
        for i, inputs in tqdm(enumerate(dl)):
            inputs = {key:val.reshape(val.shape[0],-1).to(device) for key,val in inputs.items()}
            outputs = model(**inputs)
            outputs = outputs.detach().cpu().numpy()
            embeddings.extend(outputs)
    return np.array(embeddings)


train_embeddings1 =  get_embeddings(train_data,'../input/clr-roberta/model0/model0.bin')
test_embeddings1 = get_embeddings(test_data,'../input/clr-roberta/model0/model0.bin')

train_embeddings2 =  get_embeddings(train_data,'../input/clr-roberta/model1/model1.bin')
test_embeddings2 = get_embeddings(test_data,'../input/clr-roberta/model1/model1.bin')

train_embeddings3 =  get_embeddings(train_data,'../input/clr-roberta/model2/model2.bin')
test_embeddings3 = get_embeddings(test_data,'../input/clr-roberta/model2/model2.bin')

train_embeddings4 =  get_embeddings(train_data,'../input/clr-roberta/model3/model3.bin')
test_embeddings4 = get_embeddings(test_data,'../input/clr-roberta/model3/model3.bin')

train_embeddings5 =  get_embeddings(train_data,'../input/clr-roberta/model4/model4.bin')
test_embeddings5 = get_embeddings(test_data,'../input/clr-roberta/model4/model4.bin')
from sklearn.linear_model import Ridge
def get_preds_svm(X,y,X_test,RidgeReg=0,bins=bins,nfolds=5,C=20,kernel='rbf'):
    scores = list()
    preds = np.zeros((X_test.shape[0]))
    
    kfold = StratifiedKFold(n_splits=config['nfolds'],shuffle=True,random_state=config['seed'])
    for k, (train_idx,valid_idx) in enumerate(kfold.split(X,bins)):
        if(RidgeReg):
            print("ridge...")
            model = Ridge(alpha=50.0)
        else:
            model = SVR(C=C,kernel=kernel,gamma='auto')
        X_train,y_train = X[train_idx], y[train_idx]
        X_valid,y_valid = X[valid_idx], y[valid_idx]
        
        model.fit(X_train,y_train)
        prediction = model.predict(X_valid)
        score = rmse_score(prediction,y_valid)
        print(f'Fold {k} , rmse score: {score}')
        scores.append(score)
        preds += model.predict(X_test)
        
    print("mean rmse",np.mean(scores))
    return np.array(preds)/nfolds


svm_preds1 = get_preds_svm(train_embeddings1,target,test_embeddings1)
svm_preds2 = get_preds_svm(train_embeddings2,target,test_embeddings2)
svm_preds3 = get_preds_svm(train_embeddings3,target,test_embeddings3)
svm_preds4 = get_preds_svm(train_embeddings4,target,test_embeddings4)
svm_preds5 = get_preds_svm(train_embeddings5,target,test_embeddings5)

ridge_preds1 = get_preds_svm(train_embeddings1,target,test_embeddings1,RidgeReg=1)
ridge_preds2 = get_preds_svm(train_embeddings2,target,test_embeddings2,RidgeReg=1)
ridge_preds3 = get_preds_svm(train_embeddings3,target,test_embeddings3,RidgeReg=1)
ridge_preds4 = get_preds_svm(train_embeddings4,target,test_embeddings4,RidgeReg=1)
ridge_preds5 = get_preds_svm(train_embeddings5,target,test_embeddings5,RidgeReg=1)


#all_preds = (svm_preds1 + svm_preds2 + svm_preds3 + svm_preds4 + svm_preds5+ridge_preds1+ridge_preds2+ridge_preds3+ridge_preds4+ridge_preds5)/10
all_preds = (svm_preds1 + svm_preds2 + svm_preds3 + svm_preds4 + svm_preds5)/5

sample.target = all_preds
#sample.to_csv('submission.csv',index=False)

# model 5 : https://www.kaggle.com/abhishek/fork-of-fork-of-yum-yum-yum-93f968

In [ ]:


import torch
import pandas as pd
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer

class Dataset:
    def __init__(self, excerpt, tokenizer, max_len):
        self.excerpt = excerpt
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.excerpt)

    def __getitem__(self, item):
        text = str(self.excerpt[item])
        inputs = self.tokenizer(
            text, 
            max_length=self.max_len, 
            padding="max_length", 
            truncation=True
        )

        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]

        return {
            "input_ids": torch.tensor(ids, dtype=torch.long),
            "attention_mask": torch.tensor(mask, dtype=torch.long),
        }
    
    
def generate_predictions(model_path, max_len):
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    model.to("cuda")
    model.eval()
    
    df = pd.read_csv("../input/commonlitreadabilityprize/test.csv")
    
    dataset = Dataset(excerpt=df.excerpt.values, tokenizer=tokenizer, max_len=max_len)
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=32, num_workers=4, pin_memory=True, shuffle=False
    )

    final_output = []

    for b_idx, data in enumerate(data_loader):
        with torch.no_grad():
            for key, value in data.items():
                data[key] = value.to("cuda")
            output = model(**data)
            output = output.logits.detach().cpu().numpy().ravel().tolist()
            final_output.extend(output)
    
    torch.cuda.empty_cache()
    return np.array(final_output)


preds1 = generate_predictions("../input/a81653/", max_len=256)
preds2 = generate_predictions("../input/a81656/", max_len=256)
preds3 = generate_predictions("../input/a81657/", max_len=256)
preds4 = generate_predictions("../input/a81660/", max_len=256)
preds5 = generate_predictions("../input/a81675/", max_len=192)
preds6 = generate_predictions("../input/a87832/", max_len=256)

#https://www.kaggle.com/ankur310794/crp-mix-6-models-inference

weights_pos = [2.29301865e-08 ,9.18492143e-02 ,3.56011564e-01, 5.34926853e-09,
 8.52853500e-02 ,4.66853844e-01]

weights = weights_pos
preds = preds1*weights[0] + preds2*weights[1] + preds3*weights[2]+ preds4*weights[3] + preds5*weights[4]+ preds6*weights[5]

#preds = (preds1 + preds2 + preds3 + preds4 + preds5 + preds6) / 6

submission = pd.read_csv("../input/commonlitreadabilityprize/sample_submission.csv")
submission.target = preds
#submission.to_csv("submission.csv", index=False)

# model 6 : https://www.kaggle.com/datafan07/eda-simple-bayesian-ridge-with-sentence-embeddings

In [ ]:
# adding wordcloud in offline mode
import sys
sys.path.append('../input/wcloud/word_cloud-master')
sys.path.append('../input/sentence-transformers/sentence-transformers-master')
import wordcloud
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge
from scipy import stats
from scipy.stats import norm

import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator

# setting some globl config

plt.style.use('ggplot')
orange_black = [
    '#fdc029', '#df861d', '#FF6347', '#aa3d01', '#a30e15', '#800000', '#171820'
]
plt.rcParams['figure.figsize'] = (16,9)
plt.rcParams["figure.facecolor"] = '#FFFACD'
plt.rcParams["axes.facecolor"] = '#FFFFE0'
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.color"] = orange_black[3]
plt.rcParams["grid.alpha"] = 0.5
plt.rcParams["grid.linestyle"] = '--'



import warnings
warnings.filterwarnings("ignore")

# loding train and test data

train_df = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test_df = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

# dropping some columns

train_df=train_df[['id','excerpt','target','standard_error']]

# creating corpus

from nltk import FreqDist
from nltk import word_tokenize
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))

new = train_df['excerpt'].str.split()
new = new.values.tolist()
corpus = [word.lower() for i in new for word in i if word.lower() not in stopwords]
          

common_words = [i[0] for i in FreqDist(corpus).most_common(20)]
common_words_count = [i[1] for i in FreqDist(corpus).most_common(20)]

import sentence_transformers
from sentence_transformers import SentenceTransformer, models

model_path = '../input/finetuned-model1/checkpoint-568'
word_embedding_model = models.Transformer(model_path, max_seq_length=275)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

X_train = model.encode(train_df.excerpt, device='cuda')
X_test = model.encode(test_df.excerpt, device='cuda')

from sklearn.model_selection import StratifiedKFold
from datetime import datetime
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor

train_df['Character Count'] = train_df['excerpt'].apply(lambda x: len(str(x)))
preds = []


df_oof=train_df.copy()
df_oof['oof'] = 0

feature_importance_df = pd.DataFrame()


skf = StratifiedKFold(10, shuffle=True, random_state=42)

splits = list(skf.split(X=X_train, y=train_df['Character Count']))
for i, (train_idx, val_idx) in enumerate(splits):
    print(f'\n------------- Training Fold {i + 1} / {10}')
    print("Current Time =", datetime.now().strftime("%H:%M:%S"))
    r1 = LinearRegression()
    r2 = RandomForestRegressor(n_estimators=30, random_state=43)
    ridge = Ridge(alpha=50.0)
    br = BayesianRidge(n_iter=30, verbose=True)

    clf =   BayesianRidge(n_iter=30, verbose=True) #VotingRegressor([('r2', r2), ('br', br)])
    clf.fit(X_train[train_idx],train_df.target[train_idx])
    
    preds.append(clf.predict(X_test))
    x=clf.predict(X_train[val_idx])
    df_oof['oof'].iloc[val_idx]+= x

print(f'Training score: {mean_squared_error(train_df.target, clf.predict(X_train), squared=False)}')
print(f'OOF score across folds: {mean_squared_error(df_oof.target, df_oof.oof, squared=False)}')

# getting mean prediction across 5 folds
y_pred = np.mean(preds,0)

# creating submission csv

mysub = test_df[["id"]].copy()
mysub["target"] = y_pred





# Model 7 : https://www.kaggle.com/maunish/clrp-pytorch-roberta-inference

In [ ]:
import os
import gc
import sys
import cv2
import math
import time
import tqdm
import random
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold,StratifiedKFold

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader,Sampler

import multiprocessing
import more_itertools


from transformers import (AutoModel, AutoTokenizer, AutoModelForSequenceClassification)

from colorama import Fore, Back, Style
y_ = Fore.YELLOW
r_ = Fore.RED
g_ = Fore.GREEN
b_ = Fore.BLUE
m_ = Fore.MAGENTA
c_ = Fore.CYAN
sr_ = Style.RESET_ALL

train_data = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test_data = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
sample1 = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')

config = {
    'learning_rate':2e-5,
    'batch_size':32,
    'epochs':10,
    'nfolds':5,
    'seed':42,
    'max_len':256,
}

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(seed=config['seed'])

class CLRPDataset(Dataset):
    def __init__(self,df,tokenizer):
        self.excerpt = df['excerpt'].to_numpy()
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],return_tensors='pt',
                                max_length=config['max_len'],
                                padding='max_length',truncation=True)
        return encode
    
    def __len__(self):
        return len(self.excerpt)
class AttentionHead(nn.Module):
    def __init__(self, in_features, hidden_dim, num_targets):
        super().__init__()
        self.in_features = in_features
        self.middle_features = hidden_dim

        self.W = nn.Linear(in_features, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
        self.out_features = hidden_dim

    def forward(self, features):
        att = torch.tanh(self.W(features))

        score = self.V(att)

        attention_weights = torch.softmax(score, dim=1)

        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=1)

        return context_vector
    
    
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.roberta = AutoModel.from_pretrained('../input/roberta-base')    
        self.head = AttentionHead(768,768,1)
        self.dropout = nn.Dropout(0.1)
        self.linear = nn.Linear(768,1)

    def forward(self,**xb):
        x = self.roberta(**xb)[0]
        x = self.head(x)
        x = self.dropout(x)
        x = self.linear(x)
        return x


def get_prediction(df,path,device='cuda'):        
    model = Model()
    model.load_state_dict(torch.load(path))
    model.to(device)
    model.eval()

    tokenizer = AutoTokenizer.from_pretrained('../input/roberta-base')

    test_ds = CLRPDataset(df,tokenizer)
    test_dl = DataLoader(test_ds,
                        batch_size = config["batch_size"],
                        shuffle=False,
                        num_workers = 4,
                        pin_memory=True)

    predictions = list()
    for i, (inputs) in tqdm(enumerate(test_dl)):
        inputs = {key:val.reshape(val.shape[0],-1).to(device) for key,val in inputs.items()}
        outputs = model(**inputs)
        outputs = outputs.cpu().detach().numpy().ravel().tolist()
        predictions.extend(outputs)

    torch.cuda.empty_cache()
    return np.array(predictions)

pred1 = get_prediction(test_data,'../input/clr-roberta/model0/model0.bin')
pred2 = get_prediction(test_data,'../input/clr-roberta/model1/model1.bin')
pred3 = get_prediction(test_data,'../input/clr-roberta/model2/model2.bin')
pred4 = get_prediction(test_data,'../input/clr-roberta/model3/model3.bin')
pred5 = get_prediction(test_data,'../input/clr-roberta/model4/model4.bin')

predictions = (pred1 + pred2 + pred3 + pred4 + pred5)/5

sample1['target'] = predictions
#sample.to_csv('submission.csv',index=False)




In [ ]:
sub = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')

sub['target'] = ((pred_df2.mean(axis=1)) + (pred_df1.mean(axis=1))+ (pred_df3.mean(axis=1)) +sample['target']+ragdf['target']+sub0['target'])/6
sub.to_csv('submission.csv', index=False)
sub